## Examples of spatial pattern detection in genome ensembles

IF needed, install the following pip packages in the current Jupyter kernel:

In [1]:
import sys
!{sys.executable} -m pip install cdlib
!{sys.executable} -m pip install networkx
!{sys.executable} -m pip install umap-learn

Load the required libraries:

In [2]:
import numpy as np
#import umap
import umap.umap_ as umap

import cdlib
from cdlib import algorithms
import networkx as nx
from  scipy import sparse

import warnings
warnings.filterwarnings('ignore')

Functions used in the example:

In [8]:
def build_graph(XYZ):
    """
    Input: an array of (x,y,z) coordinates
    Output: the weighted adjacency matrix of the UMAP graph representation
    Parameters: n_neighbors is the most important parameter in the umap fuzzy_simplicial_set function. It will determine how sparse the final graph will be.
    """
#    umap.umap_.fuzzy_simplicial_set
    adj = umap.fuzzy_simplicial_set(
        XYZ,
        n_neighbors=5, # this parameter has to be fine-tuned
        random_state=np.random.RandomState(seed=42),
        metric='l2',
        metric_kwds={},
        knn_indices=None,
        knn_dists=None,
        angular=False,
        set_op_mix_ratio=1.0,
        local_connectivity=2.0,
        verbose=False
        )
    return adj

def build_communities(adj):
    """
    Input: the weighted graph adjacency matrix
    Output: a list of communities, each one a represented as a list object
    
    In this example we use the leiden algorithm as implemented in the cdlib library.
    """
    g = nx.from_scipy_sparse_matrix(adj) # generate a graph networkx obj
    eset = [(u, v) for (u, v, d) in g.edges(data=True)] # get list of edges from graph
    weights = [d['weight'] for (u, v, d) in g.edges(data=True)] # get list of weights from edges
    # find communities
    # in this example we use the Leiden algorithm
    leiden_coms = algorithms.leiden(g,weights=weights) # check if the algo is stochastic, in that case set rnd generator    
    return leiden_coms.communities # a list of lists of nodes

Create 100 random 3D structures (each composed of 1000 bins) and save them in the data directory as a numpy array named 'random_points.npy':

In [9]:
structures = np.random.default_rng().uniform(-100,100,(1000,3,100))
np.save('../data/random_points', structures)

Load the structures from the data directory.
The shape of the array is given by (#bins, #spatial_dim, #random_realizations).

In [10]:
structures = np.load('../data/random_points.npy')
print(structures.shape)

(1000, 3, 100)


Define some constant variable used during the example:

In [11]:
max_numb_str = 10 # the maximum numb of structure to consider. In principle all structure should be considered, but to speed up the example evaluation we can limit this value.
numb_str = structures.shape[2] # the number of random realizations
numb_loci = structures.shape[0] # the number of binned loci in each realization/structure

# From 3D structures to graphs 
Each structure is a very detailed representation of the genome spatial displacement. Not all of this information can be accessed experimentally with full precision. We then try to identify the robust component of the representation. We do this mapping each 3D structure to a corresponding topological graph, as constructed in the intermediate step of the UMAP algorithm. At a given length scale, specified by the number of nearest neighbors to each node, the graph represent the proximity information around each locus on the genome in terms of weighted edges.

# Communities
If we are interested in identifying similar groups of nodes, based on their graph representation, we have to look for nodes that share heavy-weighted edges. This problem is well known and referred to as "community detection". There are many algorithms providing different euristic approaches to community detection. In this example we use probably the most common one: the Leiden algorithm.

We loop over all structures, map them to a graph and find the graph communities (i.e. clusters of loci):

In [12]:
# Map each structure to a weighted graph
# In this example we use UMAP
comm = {} # communities are a python dictionary with structures as keys
for structure in range(numb_str)[:max_numb_str]: # for each structure
    XYZ = structures[:,:,structure] # get the x,y,z coordinates
    adj = build_graph(XYZ) # get the graph
    comm[structure] = build_communities(adj) # get the communities of the graph

# Recurrent communities
Once we have all the communities for all the structures in our ensemble, we determine the recurrent patterns:
for a given pair of loci (i,j) -in a given structure- we count how many times we see that pair belonging to the same community, over all structures. 

This provide us a frequency matrix P, similar to the HiC matrix. We can then study P, similarly to how we would study a contact matrix, having access to many more information on the single realization of each structures though. So fluctuations can be taken into account, in addition to the average informartion. 

In [13]:
# The matrix P is the analog of the HiC matrix:
# P(i,j) = # times loci (i,j) occur together 
for structure in range(numb_str)[:max_numb_str]: # for each structure
    for c in range(len(comm[structure])):        # for each community in the given structure
        if c == 0: # for the first community define the graph G
            G = nx.complete_graph(comm[structure][c])
        else: # for the other communities update G
            G.update(nx.complete_graph(comm[structure][c]))
    if structure == 0: # for the first structure define the sparse matrix P
        P = nx.to_scipy_sparse_matrix(G, nodelist=range(numb_loci))
    else: # for the other structures add to P
        P += nx.to_scipy_sparse_matrix(G, nodelist=range(numb_loci)) 
    #print(P.nnz)

# Non-negative Matrix Factorization (NMF)

A different approach, which does not rely on build the matrix P, could be the following:

1. Consider the array V(i,j,s), where i is the i-th locus, j is the j-th locus and s is the s-th 3D structure: the (i,j,s)-th entry of this array is 1 if (i,j) belong to the same community in the s-th structure

2. Reshape the 3-way array V into a 2-way array V(ij,s) 

3. Use NMF on V(ij,s) to determine it latent structural factors: V(ij,s) = \sum_k W(ij,k) * H(k,s) 

In [14]:
# Use NMF to decompose the V matrix similarly to what is done in word X document decomposition 
# The matrix V is #pairs X #structures matrix:
# V[(i,j),s] = 1 if the pair of loci (i,j) occurs in structure s

for structure in range(numb_str)[:max_numb_str]: # for each structure
    for c in range(len(comm[structure])):        # for each community in the given structure
        if c == 0: # for the first community define the graph G
            G = nx.complete_graph(comm[structure][c])
        else: # for the other communities update G
            G.update(nx.complete_graph(comm[structure][c]))
    
    if structure == 0: # for the first structure define the sparse matrix sparse_structure_2d
        node_i = [e[0] for e in G.edges]
        node_j = [e[1] for e in G.edges]
        s = [structure for e in G.edges]
        sparse_structure = sparse.coo_matrix((s, (node_i, node_j)), 
                                             shape=(numb_loci, numb_loci), dtype=np.int8)
        # reshape to 1d sparse array
        sparse_structure_1d = sparse_structure.reshape((numb_loci*numb_loci,1))
        # build the 2d edgesXstructure sparse array:
        rows = sparse_structure_1d.row # the occurring pairs
        cols = [v for v in sparse_structure_1d.data] # the occurring structures
        data = [1]*len(sparse_structure_1d.data) # 1 if loci pair is in structures
        V = sparse.coo_matrix((data, (rows, cols)),
                              shape=(numb_loci*numb_loci,max_numb_str),
                              dtype=np.int8) # define the V matrix
    else:
        node_i = [e[0] for e in G.edges]
        node_j = [e[1] for e in G.edges]
        s = [structure for e in G.edges]
        sparse_structure = sparse.coo_matrix((s, (node_i, node_j)), 
                                             shape=(numb_loci, numb_loci), dtype=np.int8)
        # reshape to 1d sparse array
        sparse_structure_1d = sparse_structure.reshape((numb_loci*numb_loci,1))
        # build the new 2d edgesXstructure sparse array:
        rows = sparse_structure_1d.row
        cols = [v for v in sparse_structure_1d.data]
        data = [1]*len(sparse_structure_1d.data)
        # concatenate old and new data
        data = np.concatenate((V.data, data))
        rows = np.concatenate((V.row, rows))
        cols = np.concatenate((V.col, cols))
        V = sparse.coo_matrix((data, (rows, cols)),
                              shape=(numb_loci*numb_loci,max_numb_str),
                              dtype=np.int8) # update the V matrix

In [15]:
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='nndsvd', random_state=0)
W = model.fit_transform(V)
H = model.components_

In [16]:
# Get the node_i and node_j labels from the rows of the V matrix
row_V_matrix = 19088
np.unravel_index([row_V_matrix],  (numb_loci,numb_loci))

(array([19]), array([88]))